In [1]:
QRY_PATH="/home/global/anaconda2/tract_querier/queries/freesurfer_queries.qry"
#SUBJ_ID="T02S01"
SUBJ_ID="p_DG21_a"
NAS="tupac"
#protocole="healthy_volunteers"
protocole="alexcis"
DIR_PATH="/NAS/"+NAS+"/protocoles/"+protocole
FS_PATH=DIR_PATH+"/FS53/"
DTI_PATH=FS_PATH+SUBJ_ID+"/dti/"
TRACTO="whole_brain_6_1500000"
#SEG = DTI_PATH+"raparc+aseg.nii"
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"
MESH = "lh.white.ras"

In [1]:
from nibabel import trackvis as tv
from dipy.viz import fvtk
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import AveragePointwiseEuclideanMetric
from dipy.segment.metric import CenterOfMassFeature
from dipy.segment.metric import EuclideanMetric
from dipy.segment.metric import IdentityFeature
from dipy.segment.clustering import QuickBundles
from dipy.io.image import load_nifti, save_nifti
from dipy.tracking.streamline import set_number_of_points
from dipy.tracking.utils import density_map
from nibabel import freesurfer as fs
from dipy.utils.optpkg import optional_package
import dipy.io.vtk as io_vtk
import dipy.viz.utils as ut_vtk
from dipy.viz import window, actor, ui
from dipy.tracking import utils
import nipype.interfaces.mrtrix as mrt
from dipy.tracking.streamline import select_random_set_of_streamlines
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import random
from dipy.tracking.streamline import transform_streamlines
from dipy.denoise.enhancement_kernel import EnhancementKernel
from dipy.tracking.fbcmeasures import FBCMeasures
from nibabel.streamlines import Field
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.viz import colormap
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular

ImportError: cannot import name gammaln

In [13]:
D33 = 1.0
D44 = 0.02
t = 1
k = EnhancementKernel(D33, D44, t)

r = fvtk.ren()

nii2 = nib.load(DTI_PATH+"raparc.a2009s+aseg.nii")
raparc = fvtk.slicer(nii2.get_data())#,nii2.affine)
print(nii2)
raparc.display(60, None, None)
#fvtk.add(r,raparc)

data_colorfa = nib.load(DTI_PATH+"dti_finalcor_FA.nii.gz")
print(data_colorfa)
dti_fa = fvtk.slicer(data_colorfa.get_data())#,data_colorfa.affine)
dti_fa.display(64, None, None)
#fvtk.add(r,dti_fa)

#Lecture de la surface
vtk, have_vtk, setup_module = optional_package('vtk')
white_mesh=fs.read_geometry(FS_PATH+SUBJ_ID+"/surf/"+MESH)
my_polydata = vtk.vtkPolyData()
test=actor.apply_affine(np.linalg.inv(nii2.affine), white_mesh[0]) 

ut_vtk.set_polydata_vertices(my_polydata, test)
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])

#transform = vtk.vtkTransform()
#transform.SetMatrix(np.reshape(data_colorfa.affine,16).tolist())
#print(transform)
#print(data_colorfa.affine)
#print(my_polydata)

#transformFilter=vtk.vtkTransformPolyDataFilter()
#transformFilter.SetTransform(transform)
#transformFilter.SetInputConnection(my_polydata)
#transformFilter.Update()

stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.6) 
#r.add(stream_actor)
#fvtk.add(r, vol_actor)

li = ['cc_1','cc_3']#,'cc_4'#,'cc_2']#,'cc_7'] 
for BUNDLE in li:
    color=(random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1))
    print(color)
    trkS = nib.streamlines.load(FS_PATH+SUBJ_ID+"/bundle_"+BUNDLE+".trk")
    #Track ouverture
    #streams, hdr = tv.read(tractogram)    
    #input_spacetmp=np.array(tv.aff_from_hdr(hdr));
    #input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])
    
    #tracto = [i[0] for i in streams]    
    #plot_tracto = select_random_set_of_streamlines(tracto, 1000)
    
    #plot_tracto=list(utils.move_streamlines(plot_tracto,input_space))
    #fvtk.add(r, fvtk.streamtube(plot_tracto,color,linewidth=0.1,opacity=0.1));
    #fvtk.add(r, fvtk.streamtube(tracto,color,linewidth=0.1,opacity=0.5));

    #Track ouverture & Affichage    
    trk = nib.streamlines.load(FS_PATH+SUBJ_ID+"/bundle_"+BUNDLE+"_clean.trk")
    #trk.header.update(trkS.header)
    #trk.header[Field.VOXEL_TO_RASMM]=trkS.header[Field.VOXEL_TO_RASMM]
    #trk.header[Field.VOXEL_SIZES]=trkS.header[Field.VOXEL_SIZES]
    #trk.header[Field.ORIGIN]=trkS.header[Field.ORIGIN]
    bundle_native3 = transform_streamlines(trkS.tractogram.streamlines,np.linalg.inv(data_colorfa.affine))
    #fvtk.add(r,actor.line(bundle_native3,fvtk.colors.white,linewidth=0.1,opacity=0.2))
    
    #bundle_native = transform_streamlines(trk.tractogram.streamlines,np.linalg.inv(data_colorfa.affine))
    #bundle_native2 = transform_streamlines(bundle_native,np.linalg.inv(data_colorfa.affine))    
    #fvtk.add(r,actor.line(bundle_native2,color,linewidth=0.2,opacity=0.5))    
    
    fbc = FBCMeasures(bundle_native3, k)
    fbc_sl_orig2, clrs_orig2, rfbc_orig2 = fbc.get_points_rfbc_thresholded(0, emphasis=0.01)
    fbc_sl_thres2, clrs_thres2, rfbc_thres2 = fbc.get_points_rfbc_thresholded(0.125, emphasis=0.01)
    fvtk.add(r, actor.line(fbc_sl_orig2, clrs_orig2, linewidth=0.2))
    fvtk.add(r, actor.line(fbc_sl_thres2, clrs_thres2, linewidth=0.2))
   
    #Track cluster
    feature = ResampleFeature(nb_points=240)
    metric = AveragePointwiseEuclideanMetric(feature=feature)  # a.k.a. MDF    
    qb = QuickBundles(threshold=100, metric=metric)
    qb.max_nb_clusters = 1
    cluster = qb.cluster(bundle_native2) 
    print("Nb. clusters:", len(cluster))
    print("Cluster sizes:", map(len, cluster))

    #clus=list(utils.move_streamlines(cluster.centroids,input_space))
    clus=list(cluster.centroids)
    #fvtk.add(r, fvtk.streamtube(clus,data_colorfa.get_data(),linewidth=1));    
     
a=fvtk.axes()
fvtk.add(r,a)
fvtk.add(r, actor.line(fbc_sl_orig, clrs_orig, linewidth=0.2))
fvtk.add(r, actor.line(fbc_sl_thres, clrs_thres, linewidth=0.2))

#fvtk.record(r, out_path=FS_PATH+SUBJ_ID+"/dti/clean.png", size=(600,600))
fvtk.show(r)

The kernel already exists. Loading from /tmp/kernel_d33@1.00_d44@0.02_t@1.00_numverts100.npy
<class 'nibabel.nifti1.Nifti1Image'>
data shape (128, 128, 66)
affine: 
[[  -2.            0.            0.          137.89001465]
 [   0.            2.            0.         -128.83609009]
 [   0.            0.            2.          -32.35711288]
 [   0.            0.            0.            1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : 
db_name         : 
extents         : 0
session_error   : 0
regular         : r
dim_info        : 0
dim             : [  3 128 128  66   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  2.  2.  2.  0.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10


IOError: [Errno 2] No such file or directory: '/NAS/tupac/protocoles/alexcis/FS53/p_DG21_a/surf/lh.white.ras'

In [10]:
tractogram=FS_PATH+SUBJ_ID+"/bundle_cc_1.trk"
trkS = nib.streamlines.load(tractogram)
tractogram=FS_PATH+SUBJ_ID+"/bundle_cc_1_clean.trk"
trkD = nib.streamlines.load(tractogram)
#trkD.header[Field.VOXEL_TO_RASMM]=trkS.header[Field.VOXEL_TO_RASMM]
#trkD.header.update(trkS.header)
print(trkS.header)
print(trkD.header)
print(trkS.tractogram.streamlines[0])
print(trkD.tractogram.streamlines[0])

{'origin': array([ 0.,  0.,  0.], dtype=float32), 'reserved': '', 'voxel_sizes': array([ 2.,  2.,  2.], dtype=float32), 'property_name': array(['', '', '', '', '', '', '', '', '', ''],
      dtype='|S20'), 'invert_y': '', 'invert_x': '', 'invert_z': '', 'dimensions': array([128, 128,  66], dtype=int16), 'swap_yz': '', 'magic_number': 'TRACK', 'nb_scalars_per_point': 0, 'version': 2, 'endianness': '<', 'hdr_size': 1000, 'pad1': '', 'voxel_order': 'LAS', 'pad2': '', '_offset_data': 1000, 'nb_properties_per_streamline': 0, 'voxel_to_rasmm': array([[  -2.        ,    0.        ,    0.        ,  137.89001465],
       [   0.        ,    2.        ,    0.        , -128.83609009],
       [   0.        ,    0.        ,    2.        ,  -32.35711288],
       [   0.        ,    0.        ,    0.        ,    1.        ]], dtype=float32), 'swap_zx': '', 'swap_xy': '', 'scalar_name': array(['', '', '', '', '', '', '', '', '', ''],
      dtype='|S20'), 'image_orientation_patient': array([ 1.,  0.,  0.

Linda Marrakchi-Kacem (Thèse 2011)
Connectivité anatomique des noyaux gris centraux : développements méthodologiques et application aux troubles moteurs

In [5]:
hue = [0.0, 0.0]  # red only
saturation = [0.0, 1.0]  # white to red
lut_cmap = actor.colormap_lookup_table(hue_range=hue,saturation_range=saturation)

data, affine = load_nifti(FS_PATH+"p_AR28_na/dti/dti.nii.gz")
bvals, bvecs = read_bvals_bvecs(FS_PATH+"p_AR28_na/dti/dti.bval",FS_PATH+"p_AR28_na/dti/dti.bvec")
gtab = gradient_table(bvals, bvecs)
tenmodel = TensorModel(gtab)
tenfit = tenmodel.fit(data)
color_fa = fvtk.slicer(tenfit.color_fa,affine)
fa = fvtk.slicer(tenfit.fa,affine)
fa.display(None, 50, None)
print(fa.shape)
print(fa)

md = fvtk.slicer(tenfit.md,affine)
rd = fvtk.slicer(tenfit.rd,affine)
ad = fvtk.slicer(tenfit.ad,affine)

#trk2 = nib.streamlines.load(FS_PATH+"p_AR28_na/dti/whole_brain_6_1500000.tck")
trk3 = nib.streamlines.load(FS_PATH+"p_AR28_na/bundle_thalamo_frontal.left.trk")
trk4 = nib.streamlines.load(FS_PATH+"p_AR28_na/bundle_cc_4.trk")
#trk = nib.streamlines.load(tractogram)
#print(trk.header)
#print(trk2.header)
#print(trk3.header)
#print(trk4.header)
#plot_tracto = select_random_set_of_streamlines(trk.tractogram.streamlines, 10000)

r = fvtk.ren()
fvtk.add(r,fa)


vtk, have_vtk, setup_module = optional_package('vtk')
white_mesh=fs.read_geometry(FS_PATH+"p_AR28_na/surf/lh.white.ras")
my_polydata = vtk.vtkPolyData()
#print(white_mesh.shape)
#test=actor.apply_affine([[1,0,0,0][0,1,0,0][0,0,1,0][0,0,0,1]], white_mesh[0]) 

ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])

stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(0.7) 
r.add(stream_actor)

nii2 = nib.load(FS_PATH+"p_AR28_na/dti/t1_dti_ras.nii")
raparc2 = fvtk.slicer(nii2.get_data(),nii2.affine)
raparc2.display(120, None, None)
fvtk.add(r,raparc2)

nii = nib.load(FS_PATH+"p_AR28_na/dti/dti_finalcor_FA.nii.gz")
raparc = fvtk.slicer(nii.get_data(),nii.affine)
raparc.display(60, None, None)
fvtk.add(r,raparc)

#test=actor.apply_affine(nii.affine,nii.get_data()) 

fvtk.add(r, actor.line(plot_tracto,raparc2.display(None, None, None),linewidth=0.1,opacity=0.3));
#bundle_native = transform_streamlines(trk4.tractogram.streamlines,np.linalg.inv(nii.affine))

#test = density_map(trk4.streamlines,fa.shape,affine=nii.affine)
#print(test.shape)
#print(type(test))
#fvtk.add(r, actor.line(trk4.streamlines,linewidth=0.1,opacity=0.3,lookup_colormap=lut_cmap));
#fvtk.add(r, actor.line(trk3.streamlines,fa.display(None, None, None),linewidth=0.1,opacity=0.3,lookup_colormap=lut_cmap));

bar = actor.scalar_bar()
#fvtk.add(r,bar)
a=fvtk.axes()
fvtk.add(r,a)

#fvtk.add(r,fvtk.streamtube(trk3.streamlines,raparc.get,linewidth=1))

#fvtk.add(r, actor.line(fbc_sl_orig, clrs_orig, linewidth=0.2))
#fvtk.add(r, actor.line(fbc_sl_thres, clrs_thres, linewidth=0.2))
fvtk.show(r)
#fvtk.record(r, out_path=FS_PATH+"p_AR28_na/dti/orient.png", size=(600,600))

(128, 128, 66)
vtkImageActor (0x4973440)
  Debug: Off
  Modified Time: 11182
  Reference Count: 1
  Registered Events: (none)
  Dragable: On
  Pickable: On
  AllocatedRenderTime: 10
  EstimatedRenderTime: 0
  NumberOfConsumers: 0
  RenderTimeMultiplier: 1
  Visibility: On
  PropertyKeys: none.
  useBounds: 1
  IsIdentity: true
  Position: (0, 0, 0)
  Orientation: (0, 0, 0)
  Origin: (0, 0, 0)
  Scale: (1, 1, 1)
  Bounds: 
    Xmin,Xmax: (-125.658, 128.342)
    Ymin,Ymax: (-29.9284, -29.9284)
    Zmin,Zmax: (-58.1077, 71.8923)
  UserTransform: (none)
  UserMatrix: (none)
  Property:
    Debug: Off
    Modified Time: 10351
    Reference Count: 1
    Registered Events: (none)
    ColorWindow: 255
    ColorLevel: 127.5
    UseLookupTableScalarRange: Off
    LookupTable: 0
    Opacity: 1
    Ambient: 1
    Diffuse: 0
    InterpolationType: Linear
    LayerNumber: 0
    Checkerboard: Off
    CheckerboardSpacing: 10 10
    CheckerboardOffset: 0 0
    Backing: Off
    BackingColor: 0 0 0
  Map

NameError: name 'plot_tracto' is not defined

In [8]:
print(tenfit)
print(np.histogram(tenfit.fa))
print(np.histogram(tenfit.ad))
print(np.histogram(tenfit.rd))
print(np.histogram(tenfit.md))
test = density_map(trk4.streamlines,fa.shape,affine=fa.header.affine)
print(test)

(array([592033,  92785,  82417,  57552,  40175,  27619,  19204,  44022,
        34208,  91329]), array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]))
(array([1081245,      51,      22,       4,       3,       3,       4,
             5,       4,       3]), array([  1.00001000e-09,   1.12731393e+00,   2.25462787e+00,
         3.38194180e+00,   4.50925573e+00,   5.63656967e+00,
         6.76388360e+00,   7.89119753e+00,   9.01851146e+00,
         1.01458254e+01,   1.12731393e+01]))
(array([1081245,      51,      22,       4,       3,       3,       4,
             5,       4,       3]), array([  1.00001000e-09,   1.12717168e+00,   2.25434335e+00,
         3.38151503e+00,   4.50868671e+00,   5.63585838e+00,
         6.76303006e+00,   7.89020174e+00,   9.01737341e+00,
         1.01445451e+01,   1.12717168e+01]))
(array([1081245,      51,      22,       4,       3,       3,       4,
             5,       4,       3]), array([  1.00001000e-09,   1.12721910e+00,   2.2544

AttributeError: 'ImageActor' object has no attribute 'header'

In [6]:
from dipy.viz import actor, window
ren = window.Renderer()
lines = [np.random.rand(10, 3), np.random.rand(20, 3)]
print(np.shape(lines))
print(lines)
colors = np.random.rand(2, 3)
print(np.shape(colors))
print(colors)

c = actor.line(lines, colors)
ren.add(c)
window.show(ren)

(2,)
[array([[ 0.73586019,  0.25348496,  0.00751756],
       [ 0.3654789 ,  0.34527801,  0.23125331],
       [ 0.23033939,  0.26568569,  0.04819015],
       [ 0.89790543,  0.52363623,  0.37215476],
       [ 0.46661856,  0.94201243,  0.59212224],
       [ 0.13541595,  0.25901985,  0.0823235 ],
       [ 0.68002114,  0.23700455,  0.10204289],
       [ 0.01660428,  0.72280978,  0.78988682],
       [ 0.48623199,  0.73479642,  0.65321769],
       [ 0.5277191 ,  0.56248649,  0.8201968 ]]), array([[ 0.08475483,  0.7437414 ,  0.00692238],
       [ 0.75587658,  0.48325087,  0.74182973],
       [ 0.0595725 ,  0.85587286,  0.00601925],
       [ 0.58275506,  0.91332602,  0.16099368],
       [ 0.73346947,  0.14067628,  0.93078141],
       [ 0.63352929,  0.36834068,  0.74432742],
       [ 0.08328213,  0.76396862,  0.75605078],
       [ 0.15862997,  0.69269324,  0.93088749],
       [ 0.51672123,  0.99279346,  0.60165682],
       [ 0.43638614,  0.53129643,  0.95438798],
       [ 0.47743129,  0.55434909

In [7]:
print("tenfit.fa")
print(trk4.affine)
print(fa_img.affine)
print(affine)
print(np.shape(FA))
print(np.shape(affine))
print(FA[10,10,10])
#print(np.histogram(fa_img.get_data()))
#print(np.histogram(tenfit.fa))
#print(fa.display(20,None,None))

print(type(fa_img.get_data()))
print(type(test))

tenfit.fa


NameError: name 'trk4' is not defined

In [4]:
trk4 = nib.streamlines.load(FS_PATH+"p_AR28_na/bundle_cc_4.trk")
import vtk as vtk

#fa = fvtk.slicer(tenfit.fa,affine)

#FA = fractional_anisotropy(tenfit.evals)
#FA[np.isnan(FA)] = 0
#fa_img = nib.Nifti1Image(FA.astype(np.float32),affine)

#ad = fvtk.slicer(tenfit.ad,affine)
#v = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0],[0, 0, 0, 1]])
#c = colormap.orient2rgb(v)

#r = fvtk.ren()
#test = fvtk.slicer(fa_img.get_data(),affine)
#test.display(None, None, None)

#test2=actor.apply_affine(affine,FA) 

trk_shifted = transform_streamlines(trk4.tractogram.streamlines,np.linalg.inv(affine))
bundle_native = actor.line(trk_shifted,tenfit.fa,linewidth=0.1,opacity=0.3);
fvtk.add(r,bundle_native);
print(bundle_native.GetBounds())
print(bundle_native.GetPosition())


bundle=actor.line(trk4.streamlines,tenfit.fa,linewidth=0.1,opacity=0.3)
fvtk.add(r,bundle);
print(bundle.GetBounds())
print(bundle.GetPosition())

#transform = vtk.vtkTransform()
#transform_matrix = vtk.vtkMatrix4x4()
#transform_matrix.DeepCopy(np.reshape(affine,16).tolist())
#transform.SetMatrix(transform_matrix)

print(type(bundle_native))
print(type(bundle_native.GetActors))

transformFilter=vtk.vtkTransformPolyDataFilter()
transformFilter.SetTransform(transform)
transformFilter.SetInputConnection(bundle_native)
transformFilter.Update()

actor.SetTransform(transform)
fvtk.add(r,bundle_native);

#fvtk.add(r,);
fvtk.add(r,actor.scalar_bar())
fvtk.add(r,fvtk.axes())
fvtk.show(r)

NameError: name 'affine' is not defined